That's how we are to calculate resonator's characteristics:

In [66]:
# scale factors
def freq_m(freq):
    freq *= 1e3; return freq;
def cap_m(cap):
    cap /= 1e9; return cap;

def inductance(freq,cap):
    return 1/(freq_m(freq)**2 * cap_m(cap))

def quality_factor(freq,cap,res):
    return 1/(freq_m(freq) * cap_m(cap) * res)

def reactive_res(freq,cap):
    return 1/(freq_m(freq)*cap_m(cap))

def resistance_sum(rho,q):
    return rho/q
    #return (ind/cap_m(cap)*res_l) * sqrt( 1 + (res_l/(freq_m(freq)*ind)**2 ))
                                         
def resistance_smax(rho):
    return rho*1e-3
                                         
def current(e_voltage,res_s):
    return e_voltage/res_s

def resistance_ind(res,res_sum,res_s):
    return res_sum - res - res_s

And now gonna check out what's going on in our circuit...

In [70]:
import numpy as np
import math
data = np.loadtxt('data_exp.tsv', delimiter='\t')

out = open("data.tsv",'w')
for n in range(7):
    cap = data[n][0]
    freq = data[n][1]*2*math.pi
    u = data[n][2]
    e = data[n][3]
    r = 3.5 # Ohm
    
    ind = inductance(freq,cap)
    q = quality_factor(freq,cap,r)
    rho = reactive_res(freq,cap)
    r_sum = resistance_sum(rho,q)
    r_s = resistance_smax(rho)
    r_l = resistance_ind(r,r_sum,r_s)
    i = current(e,r_s)
    
    out.write(str(n+1)+'\t')
    for i in range(4):
        out.write(str(data[n][i])+'\t')
    out.write("%0.2f\t%0.2f\t%0.2f\t%0.2f\t%0.2f\t%0.2f\t%0.2f\n" % (ind*1e6,q,rho,r_sum,r_s,r_l,i*1e3))
    
out.close()